<a href="https://colab.research.google.com/github/MKolaksazov/Data-Science/blob/main/Colab%20code%20(graphs)/Tukey_test_letters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import os

# Get the list of files in the current directory
files = os.listdir('.')

# Find the CSV file
csv_file = [file for file in files if file.endswith('.csv')][0]

# прочитане директно от изхода на приложението за обработка на сурови данни от апарата
df = pd.read_csv(csv_file, delimiter=',', decimal='.', skiprows=0)
csv_file

'Ms_2509_bca_all.csv'

In [54]:
method = 'AO'

group_order = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']

In [55]:
df.head()

,groups,method,rep1,rep2
0,n1,GOX,1.790984,1.659836
1,n2,GOX,1.028689,1.020492
2,n3,GOX,1.250000,1.299180
3,n4,GOX,1.381148,1.217213
4,n5,GOX,1.364754,1.282787


In [56]:
def rep_to_numeric(d):
  # Convert 'rep1' and 'rep2' columns to numeric, handling errors
  d['rep1'] = pd.to_numeric(d['rep1'], errors='coerce')
  d['rep2'] = pd.to_numeric(d['rep2'], errors='coerce')

  # Check for missing values in 'rep1' and 'rep2'
  print(d.isnull().sum())
  d = d.dropna();
  print(d.isnull().sum())

  return d

In [57]:
def methods_to_new_cols():
  for method in df_long['method'].unique():
    df_long[method] = df_long[df_long['method'] == method]['value']
    print ('ready')

In [58]:
for col in df.columns:
  if col == 'rep1':
    df = rep_to_numeric(df)
    df_long = pd.melt(df, id_vars=['groups', 'method'], value_vars=['rep1', 'rep2'], var_name='replicate', value_name='value')
    methods_to_new_cols()

groups    0
method    0
rep1      0
rep2      0
dtype: int64
groups    0
method    0
rep1      0
rep2      0
dtype: int64
ready
ready
ready
ready
ready


In [59]:
display(df_long.head())

,groups,method,replicate,value,GOX,PM,FRAP,MDA,TPC
0,n1,GOX,rep1,1.790984,1.790984,NaN,NaN,NaN,NaN
1,n2,GOX,rep1,1.028689,1.028689,NaN,NaN,NaN,NaN
2,n3,GOX,rep1,1.250000,1.250000,NaN,NaN,NaN,NaN
3,n4,GOX,rep1,1.381148,1.381148,NaN,NaN,NaN,NaN
4,n5,GOX,rep1,1.364754,1.364754,NaN,NaN,NaN,NaN


In [60]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.stats.multicomp as mc

In [61]:
df_filtered = df_long[(df_long['method'] == 'PM') | (df_long['method'] == 'FRAP')]
df_filtered['groups_method'] = df_filtered['groups'] + ' ' + df_filtered['method']

/tmp/ipython-input-3730032843.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['groups_method'] = df_filtered['groups'] + ' ' + df_filtered['method']


In [62]:
if method == 'AO':
  data = pd.DataFrame({
      'Variant': df_filtered['groups_method'],
      'Value': df_filtered['value']
  })

else:
  data = pd.DataFrame({
      'Variant': df_long.pop('groups'),
      'Value': df_long.pop(method)
  })

data.dropna(inplace=True)

In [64]:
display(data)

,Variant,Value
9,n1 PM,61.800987
10,n2 PM,61.554276
11,n3 PM,63.281250
12,n4 PM,70.559211
13,n5 PM,69.202303
14,n6 PM,45.394737
15,n7 PM,50.945724
16,n8 PM,54.399671
17,n9 PM,53.166118
18,n1 FRAP,59.176301


In [65]:
# define columns for data and then grouping
col_to_group='Variant'
col_for_data='Value'

#Now take teh data and regroup for anova
samples = [cols[1] for cols in data.groupby(col_to_group)[col_for_data]]

# Предполагаме, че samples е списък от pandas Series
# Пример: samples = [pd.Series([1, 2]), pd.Series([3, 4, 5]), pd.Series([6])]

# Извличаме всички стойности от всички Series в един плосък списък
all_values = []
for series in samples:
    all_values.extend(series.tolist())

# Конвертираме плоския списък в numpy масив
samples = np.array(all_values, dtype='float32')

print(samples.shape, data.shape, data)


#I am not sure how this works but it makes an numpy array for each group
samples = np.asarray(samples).astype('float')
f_val, p_val = stats.f_oneway(*samples)
# I am not sure what this star does but this passes all the numpy arrays correctly
#print(samples)
# this if statement can be uncommmented if you don't won't to go furhter with out p<0.05
#if p_val<0.05:    #If the p value is less than 0.05 it then does the tukey
mod = mc.MultiComparison(data[col_for_data].astype('float'), data[col_to_group])
thsd=mod.tukeyhsd()

#this is a function to do Piepho method.
#AN Alogrithm for a letter based representation of al-pairwise comparisons.
tot=len(thsd.groupsunique)
#print(thsd.groupsunique)
#make an empty dataframe that is a square matrix of size of the groups. #set first column to 1
df_ltr=pd.DataFrame(np.nan, index=np.arange(tot),columns=np.arange(tot))
#df_ltr.iloc[:,0]=1
df_ltr[df_ltr.columns[0]] = 1
#df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)

count=0
#df_nms = pd.DataFrame('', index=np.arange(tot), columns=['names'])

# I make a dummy dataframe to put axis labels into.  sd stands for signifcant difference

for i in np.arange(tot):   #I loop through and make all pairwise comparisons.
    for j in np.arange(i+1,tot):
        if thsd.reject[count]:
            for cn in np.arange(tot):
                #print('i=',i,'j=',j,'cn=', cn, df_ltr)
                if df_ltr.iloc[i,cn]==1 and df_ltr.iloc[j,cn]==1:
                  #If the column contains both i and j shift and duplicat
                    df_ltr=pd.concat([df_ltr.iloc[:,:cn+1],df_ltr.iloc[:,cn+1:].T.shift().T],axis=1)
                    #print('i=',i,'j=',j,'cn=', cn, df_ltr)
                    df_ltr[df_ltr.columns[cn+1]]=df_ltr[df_ltr.columns[cn]]
                    df_ltr.iloc[i,cn]=0
                    df_ltr.iloc[j,cn+1]=0
                    print('i=',i,'j=',j,'cn=', cn, df_ltr)
                #Now we need to check all columns for abosortpion.
                for cleft in np.arange(len(df_ltr.columns)-1):
                    for cright in np.arange(cleft+1,len(df_ltr.columns)):
                        if (df_ltr[df_ltr.columns[cleft]].isna()).all()==False and (df_ltr[df_ltr.columns[cright]].isna()).all()==False:
                            if (df_ltr[df_ltr.columns[cleft]]>=df_ltr[df_ltr.columns[cright]]).all()==True:
                                print('ri=',i,'j=',j,'cn=', cn, "cright", cright, df_ltr)
                                df_ltr[df_ltr.columns[cright]]=0

                                df_ltr=pd.concat([df_ltr[df_ltr.columns[:cright]],df_ltr[df_ltr.columns[cright:]].T.shift(-1).T],axis=1)
                                print('ri=',i,'j=',j,'cn=', cn, "cright", cright, df_ltr)
                            if (df_ltr[df_ltr.columns[cleft]]<=df_ltr[df_ltr.columns[cright]]).all()==True:
                                df_ltr[df_ltr.columns[cleft]]=0
                                print('li=',i,'j=',j,'cn=', cn, "cleft", cleft, df_ltr)
                                df_ltr=pd.concat([df_ltr[df_ltr.columns[:cleft]],df_ltr[df_ltr.columns[cleft:]].T.shift(-1).T],axis=1)
                                print('li=',i,'j=',j,'cn=', cn, "cleft", cleft, df_ltr)

        count+=1
print ("after loop: ", df_ltr)
#I sort so that the first column becomes A
df_ltr=df_ltr.sort_values(by=list(df_ltr.columns),axis=1,ascending=False)
print ("after sorting: ", df_ltr)
# I assign letters to each column
for cn in np.arange(len(df_ltr.columns)):
    df_ltr[df_ltr.columns[cn]]=df_ltr[df_ltr.columns[cn]].replace(1,chr(97+cn))
    df_ltr[df_ltr.columns[cn]]=df_ltr[df_ltr.columns[cn]].replace(0,'')
    df_ltr[df_ltr.columns[cn]]=df_ltr[df_ltr.columns[cn]].replace(np.nan,'')
    #print(df_ltr.columns[cn])

#I put all the letters into one string
df_ltr=df_ltr.astype(str)
df_ltr.sum(axis=1)
df_ltr



(36,) (36, 2)     Variant      Value
9     n1 PM  61.800987
10    n2 PM  61.554276
11    n3 PM  63.281250
12    n4 PM  70.559211
13    n5 PM  69.202303
14    n6 PM  45.394737
15    n7 PM  50.945724
16    n8 PM  54.399671
17    n9 PM  53.166118
18  n1 FRAP  59.176301
19  n2 FRAP  57.008671
20  n3 FRAP  65.028902
21  n4 FRAP  62.752890
22  n5 FRAP  63.619942
23  n6 FRAP  40.209538
24  n7 FRAP  52.565029
25  n8 FRAP  57.767341
26  n9 FRAP  50.830925
54    n1 PM  62.047697
55    n2 PM  60.197368
56    n3 PM  63.898026
57    n4 PM  71.546053
58    n5 PM  76.603618
59    n6 PM  53.166118
60    n7 PM  52.425987
61    n8 PM  57.483553
62    n9 PM  51.562500
63  n1 FRAP  49.421965
64  n2 FRAP  52.348266
65  n3 FRAP  60.043353
66  n4 FRAP  58.634393
67  n5 FRAP  67.630058
68  n6 FRAP  40.209538
69  n7 FRAP  57.008671
70  n8 FRAP  50.939306
71  n9 FRAP  47.687861


/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:573: SmallSampleWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  if is_too_small(samples, kwds):
/tmp/ipython-input-2717713097.py:24: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  f_val, p_val = stats.f_oneway(*samples)


Предаденият резултат е скъсен до последните 5000 реда.
4  NaN  
5  NaN  
6  NaN  
7  NaN  
8  NaN  
9  NaN  
10 NaN  
11 NaN  
12 NaN  
13 NaN  
14 NaN  
15 NaN  
16 NaN  
17 NaN  
li= 5 j= 16 cn= 2 cleft 2     0    1    2    3    4    5   6   7   8   9   10  11  12  13  14  15  16  \
0    0  0.0  0.0  1.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
1    0  1.0  1.0  1.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
2    0  0.0  0.0  1.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
3    0  1.0  1.0  1.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
4    0  0.0  1.0  0.0  0.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
5    0  0.0  1.0  0.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
6    1  1.0  1.0  1.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
7    1  1.0  1.0  0.0  0.0  0.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
8    1  1.0  1.0  1.0  1.0  1.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   
9  

,4,3,2,1,0,5,6,7,8,9,10,11,12,13,14,15,16,17
0,a,b,c,,,,,,,,,,,,,,,
1,a,b,c,d,,,,,,,,,,,,,,
2,a,b,c,,,,,,,,,,,,,,,
3,a,b,c,d,,,,,,,,,,,,,,
4,a,b,,d,,,,,,,,,,,,,,
5,a,b,,d,,,,,,,,,,,,,,
6,a,b,c,d,,,,,,,,,,,,,,
7,,,,d,,,,,,,,,,,,,,
8,a,,,d,,,,,,,,,,,,,,
9,,,,d,,,,,,,,,,,,,,


In [67]:
group_order = data.Variant.unique()

In [68]:
new_df_ltr = np.array(df_ltr.sum(axis=1))
new_ltr = []
# ORDERING OF THE LETTERS!!!
for i in range(len(group_order)):
  ltr = [group_order[i], thsd.groupsunique.tolist().index(group_order[i]),
  new_df_ltr[thsd.groupsunique.tolist().index(group_order[i])]]
  print(ltr)
  new_ltr.append(ltr[2])
print(np.array(df_ltr.sum(axis=1)), new_ltr)

['n1 PM', 1, 'abcd']
['n2 PM', 3, 'abcd']
['n3 PM', 5, 'abd']
['n4 PM', 7, 'd']
['n5 PM', 9, 'd']
['n6 PM', 11, 'ce']
['n7 PM', 13, 'bce']
['n8 PM', 15, 'abc']
['n9 PM', 17, 'bce']
['n1 FRAP', 0, 'abc']
['n2 FRAP', 2, 'abc']
['n3 FRAP', 4, 'abd']
['n4 FRAP', 6, 'abcd']
['n5 FRAP', 8, 'ad']
['n6 FRAP', 10, 'e']
['n7 FRAP', 12, 'abc']
['n8 FRAP', 14, 'abc']
['n9 FRAP', 16, 'ce']
['abc' 'abcd' 'abc' 'abcd' 'abd' 'abd' 'abcd' 'd' 'ad' 'd' 'e' 'ce' 'abc'
 'bce' 'abc' 'abc' 'ce' 'bce'] ['abcd', 'abcd', 'abd', 'd', 'd', 'ce', 'bce', 'abc', 'bce', 'abc', 'abc', 'abd', 'abcd', 'ad', 'e', 'abc', 'abc', 'ce']
